<a href="https://colab.research.google.com/github/yeop6134/Ai_humanities/blob/main/BERT%EA%B8%B0%EB%B0%98%EB%B6%84%EB%A5%98%EB%AA%A8%EB%8D%B8_BerTokenizerFast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 %pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/인공지능인문학/BERT기반분류모델

/content/drive/MyDrive/Colab Notebooks/인공지능인문학/BERT기반분류모델


In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능인문학/데이터전처리/독립데이터/json_1레이블수동일/test데이터split/train_독립.csv', index_col = 0)
valid_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능인문학/데이터전처리/독립데이터/json_1레이블수동일/test데이터split/valid_독립.csv', index_col = 0)
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능인문학/데이터전처리/독립데이터/json_1레이블수동일/test데이터split/test_독립.csv', index_col = 0)

In [ ]:
train_data['label'].value_counts()

False    3667
True     3496
Name: label, dtype: int64

In [ ]:
valid_data['label'].value_counts()

False    517
True     436
Name: label, dtype: int64

In [ ]:
test_data['label'].value_counts()

False    528
True     425
Name: label, dtype: int64

In [ ]:
print('훈련 데이터 개수 :',len(train_data))
print('검증 데이터 개수 :',len(valid_data))
print('테스트 데이터 개수 :',len(test_data))

훈련 데이터 개수 : 7163
검증 데이터 개수 : 953
테스트 데이터 개수 : 953


In [ ]:
train_data[train_data['label'] == True].head(n=3)

,text,label
1,그러게. 저 남자봐. 어깨 진짜 좁다. 토나올거같아.,True
3,여자들은 집안일이나 하면 되는 거 아냐?,True
4,미스김 커피 좀 타와~,True


In [ ]:
train_data[train_data['label'] == False].head(n=3)

,text,label
0,불매운동 맛을 봐야 되겠네,False
2,정당 놈들의 내로남불도 내년엔 끝이야,False
5,그러니까 이제 외모 평가 그만!,False


In [ ]:
train_data[:5] # 상위 5개 출력

,text,label
0,불매운동 맛을 봐야 되겠네,False
1,그러게. 저 남자봐. 어깨 진짜 좁다. 토나올거같아.,True
2,정당 놈들의 내로남불도 내년엔 끝이야,False
3,여자들은 집안일이나 하면 되는 거 아냐?,True
4,미스김 커피 좀 타와~,True


In [ ]:
valid_data[:5] # 상위 5개 출력

,text,label
1446,징징대면 기저귀 갈아주고 분유 물려주는 갓난애기 수준에서 성장을 못했니?,False
656,종주국이 메달도 못따는 허벌이라 보급율 높을 만도 ㅋㅋㅋ,False
1776,다리는 또 왜 저렇게 코끼리 같냐,False
1202,원래 살다보면 고등학교도 자퇴하고 그런 것이죠.,True
362,내가 하는 게임 잼민이들이 많이 하나 봐,True


In [ ]:
test_data[:5]

,text,label
1604,그거 비싼긴데 문의 한번해바라,False
733,ㄹㅇ싸이코 같은 싸함이 있다…,False
1889,"여성분들, 이런 거 말고 중요한 기사들 좀 보세요",True
1272,일본 거리는 진짜 깨끗하더라,False
721,저런 거 보고 재밌다고 하는 거 자체가 니들이 정박아라는 증거다,False


# 토크나이저를 이용한 정수 인코딩

- transformers에서 제공하는 BerTokenizerFast를 사용하여 KLUE-BERT의 토크나이저를 로드한다.
- from transformers import BertTokenizerFast  
tokenizer = BertTokenizerFast.from_pretrained("사용하고자 하는 모델 이름")

In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("klue/bert-base")

In [ ]:
# 훈련 데이터와 테스트 데이터로부터 텍스트 데이터와 레이블을 별도로 분리하여 저장한다.
X_train_list = train_data['text'].tolist()
X_valid_list= valid_data['text'].tolist()
X_test_list = test_data['text'].tolist()
y_train = train_data['label'].tolist()
y_valid = valid_data['label'].tolist()
y_test = test_data['label'].tolist()

- BerTokenizerFast 토크나이저를 사용하여 정수 인코딩과 패딩을 수행한다.

In [ ]:
X_train = tokenizer(X_train_list, truncation=True, padding=True)
X_valid = tokenizer(X_valid_list, truncation=True, padding=True)
X_test = tokenizer(X_test_list, truncation=True, padding=True)

# 데이터셋 생성 및 모델 학습

In [ ]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# 데이터를 텐서플로의 데이터셋 형태로 변환
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train),
    y_train
))

valid_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_valid),
    y_valid
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_test),
    y_test
))

- transformers에서는 텍스트 분류를 위한 모델 구현체인 TFBertForSequenceClassification을 제공한다.
- from transformers import TFBertForSequenceClassification  
model = TFBertForSequenceClassification.from_pretrained("사용하고자 하는 모델 이름", num_labels=클래스의 수(분류할 종류 개수), from_pt=True)

In [ ]:
# 옵티마이저는 Adam을 사용
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2, from_pt=True)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])
# loss=model.hf_compute_loss는 크로스 엔트로피 손실 함수가 맵핑되어 있다.

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 5에포크로 훈련
model.fit(
    train_dataset.shuffle(10000).batch(32), epochs=5, batch_size=64,
    validation_data = valid_dataset.shuffle(10000).batch(64)
)

Epoch 1/5
224/224 [==============================] - 77s 342ms/step - loss: 0.0036 - accuracy: 0.9990 - val_loss: 1.1987 - val_accuracy: 0.8290
Epoch 2/5
224/224 [==============================] - 78s 346ms/step - loss: 0.0137 - accuracy: 0.9960 - val_loss: 0.6706 - val_accuracy: 0.8783
Epoch 3/5
224/224 [==============================] - 79s 351ms/step - loss: 0.0094 - accuracy: 0.9976 - val_loss: 0.9069 - val_accuracy: 0.8437
Epoch 4/5
224/224 [==============================] - 79s 352ms/step - loss: 0.0025 - accuracy: 0.9989 - val_loss: 1.3806 - val_accuracy: 0.8080
Epoch 5/5
224/224 [==============================] - 82s 366ms/step - loss: 0.0144 - accuracy: 0.9957 - val_loss: 1.2612 - val_accuracy: 0.8216


In [ ]:
# 모델저장
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/인공지능인문학/BERT기반분류모델/model/독립데이터/json_1레이블수동일/split데이터/model_BerTokenizerFast_5epoch')
tokenizer.save_pretrained('/content/drive/MyDrive/Colab Notebooks/인공지능인문학/BERT기반분류모델/model/독립데이터/json_1레이블수동일/split데이터/tokenizer_BerTokenizerFast_5epoch')

('/content/drive/MyDrive/Colab Notebooks/인공지능인문학/BERT기반분류모델/model/독립데이터/json_1레이블수동일/split데이터/tokenizer_BerTokenizerFast_5epoch/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/인공지능인문학/BERT기반분류모델/model/독립데이터/json_1레이블수동일/split데이터/tokenizer_BerTokenizerFast_5epoch/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/인공지능인문학/BERT기반분류모델/model/독립데이터/json_1레이블수동일/split데이터/tokenizer_BerTokenizerFast_5epoch/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/인공지능인문학/BERT기반분류모델/model/독립데이터/json_1레이블수동일/split데이터/tokenizer_BerTokenizerFast_5epoch/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/인공지능인문학/BERT기반분류모델/model/독립데이터/json_1레이블수동일/split데이터/tokenizer_BerTokenizerFast_5epoch/tokenizer.json')

In [ ]:
# 테스트 데이터에 대한 정확도 측정
model.evaluate(test_dataset.batch(1024))

1/1 [==============================] - 2s 2s/step - loss: 1.0920 - accuracy: 0.8478


[1.0920352935791016, 0.8478488922119141]